# Progetto Social Media Analytics

## Dataset 

Tramite il codice python sono stati scaricati i commenti più recenti di ogni video del canale youtube *Luke Barousse* (https://www.youtube.com/c/LukeBarousse). Non sono stati scaricati i commenti di risposta ad altri commenti.

Per la fase di acquisizione dati è stato utilizzato il codice *project_SMA*.

In [1]:
!pip install pyvis

In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import itertools

In [3]:
df = pd.read_csv('./eng_comments.csv')
df.head()

,Unnamed: 0,comment,comment_id,author_url,author_name,reply_count,like_count,date,vidid,total_reply_counts,vid_title,just_date
0,0,What is the average salary?,UgxQ1EqJ4CyNODDVbGh4AaABAg,http://www.youtube.com/channel/UCv8KHbiZB_8XCB...,R,0,0,2022-01-20 10:13:03+00:00,fmLPS6FBbac,0,Become a DATA ANALYST with NO degree?!? The Go...,2022-01-20
1,1,Hi. What do you recommend For stata ana spss? ...,UgzrOyzgK26L2aqZ3xN4AaABAg,http://www.youtube.com/channel/UCG0CKDnotgl_L1...,Cristina Acuña,0,0,2022-01-20 05:29:58+00:00,U4vh2EClJic,0,DON'T get a Mac for Data Science?? - MacOS Vs....,2022-01-20
2,2,"to talk about my self , i started university w...",UgxSbCkZnhvu64wrtBx4AaABAg,http://www.youtube.com/channel/UC9HXpWGFSYmI-S...,jay Joshi,0,0,2022-01-20 04:58:33+00:00,GmTD-MBea_E,0,Day in the life of a data analyst,2022-01-20
3,3,#1 skill is learning how to identify what the ...,Ugxl35Ey3w9Of63Xh1p4AaABAg,http://www.youtube.com/channel/UCFqPt4Hyfyxp0x...,Kaden,0,0,2022-01-19 21:34:04+00:00,CC66RXeTn_4,0,How I Would Learn to be a Data Analyst,2022-01-19
4,4,Buying an exorbitantly priced Mac to end up ru...,UgyrL_pX9oGEr9mBLmJ4AaABAg,http://www.youtube.com/channel/UCDwgYFlYNDJqDe...,Ameri Kano,0,0,2022-01-19 20:35:52+00:00,U4vh2EClJic,0,DON'T get a Mac for Data Science?? - MacOS Vs....,2022-01-19


In [4]:
print('Le dimensioni del dataset sono: ' + str(df.shape))
print('Nel dataset sono presenti '+ str(df.vid_title.nunique()) + ' video diversi')
print('In totale gli utenti che hanno commentato i video sono ' + str(df.author_name.nunique()))

Le dimensioni del dataset sono: (4975, 12)
Nel dataset sono presenti 65 video diversi
In totale gli utenti che hanno commentato i video sono 4017


Nel dataset non sono presenti valori mancanti:

In [5]:
df.isnull().sum() # check dei valori mancanti

Unnamed: 0            0
comment               0
comment_id            0
author_url            0
author_name           0
reply_count           0
like_count            0
date                  0
vidid                 0
total_reply_counts    0
vid_title             0
just_date             0
dtype: int64

## Costruzione del grafo

Innanzitutto vengono generate le liste *videos* e *authors* contenenti, rispettivamente, i titoli dei video dei canali youtube ed i nomi degli autori dei commenti.

In [6]:
videos = [] # lista con i titoli dei video
authors = [] # lista con gli autori dei commenti
for i in df.vid_title.unique():
  videos.append(i)
for k in df.author_name.unique():
  authors.append(k)
print(len(videos), videos) # check della numerosità
print(len(authors), authors) # check della numerosità

65 ['Become a DATA ANALYST with NO degree?!? The Google Data Analytics Professional Certificate', "DON'T get a Mac for Data Science?? - MacOS Vs. Windows", 'Day in the life of a data analyst', 'How I Would Learn to be a Data Analyst', 'Get a JOB w/ Google Data Analytics Certificate?!? (ft. Certificate Holders)', 'BEST JOBS in Data Science', 'Power BI vs Tableau - Best BI Tool', 'Top Courses to Become a Data Analyst', 'M1 Max/Pro vs M1 Macbook for Data Science.... should you?!?', 'M1 Chip is as FAST as M1 Max!!! 🤯  (13" Mac Air Vs. 14" Mac Pro) #shorts', 'Why you should NOT be a Data Analyst', 'Use THIS to showcase EXPERIENCE in Data Science', 'Install your favorite Windows app on M1 Mac - ft. Parallels', 'STOP using Spreadsheets for Everything!', 'Standout as a Data Analyst with THIS TOOL', 'Data Scientist vs Data Analyst (funny!)', 'Data Analyst vs. Engineer Vs. Scientist (funny!)', 'Python Vs R (funny!)', 'M1 Macbook Air vs Pro (8 vs 16 GB) for Data Science', 'Learn PYTHON to be a DA

Si procede ora con la costruzione del grafo vero e proprio. 
Per fare ciò è stato inizialmente creato un grafo vuoto e, successivamente:
- sono stati generati i nodi video in base alla lista *videos* creata in precedenza, definiti appartenenti al gruppo 1;
- sono stati generati i nodi autore, in base alla lista *authors* creata in precedenza, definiti appartenenti al gruppo 2;
- sono stati generati gli archi, in base al dataset di partenza, collegando così ciascun autore di un commento al relativo video commentato.

In [7]:
G = nx.Graph()
G.add_nodes_from(videos, size=40, group=1) # aggiungiamo prima i nodi video (gruppo 1)
G.add_nodes_from(authors, size=10, group=2) # aggiungiamo poi i nodi autore (gruppo 2)
for i in range(len(df)):
  G.add_edge(df['author_name'][i], df['vid_title'][i]) # colleghiamo gli utenti ai video commentati

Trattandosi di un grafo molto grande e caratterizzato da un elevato numero di nodi, per visualizzarlo si è deciso di implementare la visualizzazione per mezzo delle funzioni disponibili nel pacchetto *pyvis*.

In seguito viene riportato il link relativo a tale pacchetto: https://pyvis.readthedocs.io/en/latest/tutorial.html.

Per una facilità interpretativa, i nodi video sono stati colorati di un colore diverso (giallo) rispetto ai nodi autore (blu); sono inoltre definiti da dimensione diverse.

*N.B.: per vedere la visualizzazione è necessario scaricare il file G.html che comparirà dopo aver eseguito la shell sotto.*

In [ ]:
net = Network('500px', '500px')
net.from_nx(G)
# net.show_buttons(filter_=['physics'])
net.show('G.html')

## Nodes degree

Viene calcolato il grado dei nodi video (ovvero il numero di archi incidenti), così da vedere quali hanno un engagement maggiore e quali minore.

In seguito vengono riportati i titoli dei 5 migliori video ed i titoli dei 5 peggiori.

In [8]:
videos_degree = {}
for video in videos:
  degree = G.degree[video]
  videos_degree[video] = degree
print(videos_degree)

{'Become a DATA ANALYST with NO degree?!? The Google Data Analytics Professional Certificate': 996, "DON'T get a Mac for Data Science?? - MacOS Vs. Windows": 91, 'Day in the life of a data analyst': 109, 'How I Would Learn to be a Data Analyst': 134, 'Get a JOB w/ Google Data Analytics Certificate?!? (ft. Certificate Holders)': 257, 'BEST JOBS in Data Science': 100, 'Power BI vs Tableau - Best BI Tool': 121, 'Top Courses to Become a Data Analyst': 140, 'M1 Max/Pro vs M1 Macbook for Data Science.... should you?!?': 61, 'M1 Chip is as FAST as M1 Max!!! 🤯  (13" Mac Air Vs. 14" Mac Pro) #shorts': 25, 'Why you should NOT be a Data Analyst': 76, 'Use THIS to showcase EXPERIENCE in Data Science': 42, 'Install your favorite Windows app on M1 Mac - ft. Parallels': 106, 'STOP using Spreadsheets for Everything!': 226, 'Standout as a Data Analyst with THIS TOOL': 70, 'Data Scientist vs Data Analyst (funny!)': 152, 'Data Analyst vs. Engineer Vs. Scientist (funny!)': 70, 'Python Vs R (funny!)': 114,

In [ ]:
print(sorted(videos_degree, key=videos_degree.get, reverse=True)[:5]) # titoli dei video con più engagement
print(sorted(videos_degree, key=videos_degree.get, reverse=False)[:5]) # titoli dei video con meno engagement

['Become a DATA ANALYST with NO degree?!? The Google Data Analytics Professional Certificate', 'Google vs IBM Data Analyst Certificate - BEST Certificate for Data Analysts', 'Get a JOB w/ Google Data Analytics Certificate?!? (ft. Certificate Holders)', 'Windows on the M1 Mac - What are your options?', 'STOP using Spreadsheets for Everything!']
['Top Non-technical Skills for Business Intelligence', 'What is Business Intelligence (BI)? #shorts', 'Conditional Format Tables in Tableau (Like Excel!) - Tableau Tutorial P.5', 'How To Use Tableau Desktop Controls - Tableau Tutorial P.2', 'Calculated Fields in Tableau (Formulas & IF Statements) - Tableau Tutorial P.6']


In [ ]:
all_values = videos_degree.values()
all_values = sorted(all_values, reverse=True)
print(all_values[:5]) # 5 valori più elevati di degree dei nodi video
print(all_values[-5:]) # 5 valori più bassi di degree dei nodi video

[996, 376, 257, 236, 226]
[2, 2, 1, 1, 1]


Il video *'Become a DATA ANALYST with NO degree?!? The Google Data Analytics Professional Certificate'* è caratterizzato da un grado estremamente maggiore rispetto a quello di tutti gli altri video del canale. 

Gli ultimi 3 video sono caratterizzati da un unico commento.

## Centrality

Vengono calcolati i seguenti valori di centralità per i nodi video:

- degree centrality (numero di connessioni tra un nodo e tutti gli altri)
- closeness centrality (misura la vicinanza di un nodo a tutti gli altri della rete, non solo quelli a cui è direttamente collegato)
- betweeness centrality (misura l’importanza di un nodo nelle «comunicazioni» con gli altri nodi e 
quindi l’influenza all'interno di una rete in virtù del «controllo sulle informazioni» che passano, 
pertanto si riferisce a quanto un nodo sia di passaggio)

In [ ]:
from networkx import degree_centrality
from networkx import closeness_centrality
from networkx.algorithms.centrality.betweenness import betweenness_centrality

In [ ]:
a = degree_centrality(G)
diz_centrality = dict((k, a[k]) for k in videos if k in a)

In [ ]:
b = closeness_centrality(G, wf_improved=True)
diz_closeness = dict((k, b[k]) for k in videos if k in b)

In [ ]:
c = betweenness_centrality(G)
diz_betweenness = dict((k, c[k]) for k in videos if k in c)

Ora vengono riportati i 5 video più rilevanti, a seconda delle 3 diverse misure di centralità:

In [ ]:
print(sorted(diz_centrality, key=diz_centrality.get, reverse=True)[:5])
print(sorted(diz_closeness, key=diz_closeness.get, reverse=True)[:5])
print(sorted(diz_betweenness, key=diz_betweenness.get, reverse=True)[:5])

['Become a DATA ANALYST with NO degree?!? The Google Data Analytics Professional Certificate', 'Google vs IBM Data Analyst Certificate - BEST Certificate for Data Analysts', 'Get a JOB w/ Google Data Analytics Certificate?!? (ft. Certificate Holders)', 'Windows on the M1 Mac - What are your options?', 'STOP using Spreadsheets for Everything!']
['Become a DATA ANALYST with NO degree?!? The Google Data Analytics Professional Certificate', 'Google vs IBM Data Analyst Certificate - BEST Certificate for Data Analysts', 'Get a JOB w/ Google Data Analytics Certificate?!? (ft. Certificate Holders)', 'Windows on the M1 Mac - What are your options?', 'STOP using Spreadsheets for Everything!']
['Become a DATA ANALYST with NO degree?!? The Google Data Analytics Professional Certificate', 'Google vs IBM Data Analyst Certificate - BEST Certificate for Data Analysts', 'Windows on the M1 Mac - What are your options?', 'Get a JOB w/ Google Data Analytics Certificate?!? (ft. Certificate Holders)', 'STOP

I video coincidono; si può notare solo uno scambio per quanto riguarda la *betweeneess centrality* tra il terzo ed il quarto video. Inoltre è evidente che i video ottenuti siano gli stessi che sono caratterizzati da un grado più elevato. Da questo è possibile dedurre che, giustamente, più un video è virale, più è fondamentale per la rete e per permettere il passaggio dell'informazione all'interno della stessa.

## Community Detection - Girvan Newman

Per il rilevamento e l'analisi delle community e relative strutture all'interno della rete, è stato utilizzato il metodo di community detection basato sull'algoritmo di Girvan Newman. Tale algoritmo elimina iterativamente gli archi caratterizzati dal numero più elevato di sentieri più corti tra i nodi che passano attraverso loro. Rimuovendo tali archi uno alla volta, la rete viene divisa in parti più piccole, dette *communities*.

In [ ]:
result = nx.algorithms.community.centrality.girvan_newman(G) #, most_valuable_edge=None)
communities = next(result)
len(communities)

6

In [ ]:
communities_len = []
for i in range(len(communities)):
  communities_len.append(len(list(communities)[i]))
print(communities_len) # numerosità delle communities

[3750, 323, 3, 2, 2, 2]


Viene calcolata la modularità della rete:

In [ ]:
modularity = nx.algorithms.community.quality.modularity(G, communities=communities)

In seguito vengono riportate le community rilevate (anche graficamente):

In [ ]:
G_gn = G # grafo con le communities
for el in range(len(communities)):
  for i in range(len(list(communities[el]))):
    G_gn.nodes[list(communities[el])[i]]['group'] = el

In [ ]:
net_greedy = Network('500px', '500px')
net_greedy.from_nx(G_gn)
net_greedy.show('G_gn.html') # plot del grafo con le communities

## Community Detection - Greedy modularity based approach

Dato che il metodo di community detection provato precedentemente non ha portato a risultati soddisfacenti, si è optato per una seconda metodologia, ovvero la *Greedy modularity based approach*. 

I risultati ottenuti sono più interessanti di prima, in quanto l'algoritmo individua 29 diverse community all'interno della rete, di dimensionalità diverse. É interessante notare come i nodi video appartenenti ad una stessa comunità siano spesso caratterizzati da un filone logico/descrittivo (ad esempio una serie di video riguardanti il linguaggio di programmazione Python).

In [ ]:
communities = sorted(nx.algorithms.community.greedy_modularity_communities(G), key=len, reverse=True)
len(communities) # quante communities

29

In [ ]:
communities_len = []
for i in range(len(communities)):
  communities_len.append(len(list(communities)[i]))
print(communities_len) # numerosità delle communities

[897, 494, 330, 215, 213, 205, 182, 169, 160, 153, 125, 112, 101, 100, 99, 95, 90, 76, 66, 66, 36, 29, 29, 21, 10, 3, 2, 2, 2]


In [ ]:
G_greedy = G # grafo con le communities
for el in range(len(communities)):
  for i in range(len(list(communities[el]))):
    G_greedy.nodes[list(communities[el])[i]]['group'] = el

In [ ]:
net_greedy = Network('500px', '500px')
net_greedy.from_nx(G_greedy)
# net.show_buttons(filter_=['physics'])
net_greedy.show('G_greedy.html') # plot del grafo con le communities

In [ ]:
modularity_greedy = nx.algorithms.community.quality.modularity(G, communities=communities)
modularity_greedy # opzionale

0.7770671329972606

Vedendo che i video caratterizzati da uno stesso macro argomento (Python ad esempio) sono caratterizzati da una serie di interazioni che ne permettono l'individuazione come comunità, risulta interessante condurre un'ulteriore Social Network Analisi, così da riuscire a carpire più informazione possibile sull'argomento d'interesse.

## Community detection for the videos in the Python playlist

Si procede ora con la seconda Social Network Analysis. I nodi sono rappresentati solamente dagli utenti che hanno commentato i video delle playlist *Python Basics* e *Python Setup*, mentre gli archi sono i video appartenenti a tali playlist. In questo modo gli utenti che hanno commentato uno o più stessi video risultano collegati da un arco caratterizzato da un peso proporzionale al numero di video con cui entrambi hanno interagito.

Tale procedura potrebbe essere estesa all'intero dataset (tutti i video dunque), ma i tempi computazionali risulterebbero molto pesanti. Per questo motivo, si è deciso di effettuare questa analisi su due playlist collegate e definite da un argomento comune, ovvero Python, così da verificare se e come gli utenti rimangono attivi nell'arco di produzione di una serie di video.

I nomi specifici delle playlist analizzate sono *Python Basics*, *Python Setup* e sono visibili al seguente link Youtube: https://www.youtube.com/c/LukeBarousse/playlists.

Viene generato un nuovo dataset, con un video per ogni riga ed una colonna *authors*, variabile caratterizzata da una lista contenente tutti gli autori che hanno commentato il relativo video.

Il dataset *new* risulta dunque costituito da 65 righe (ovvero i video del canale Youtube) e 2 colonne: titolo del video e lista di persone che hanno commentato il relativo video.

In [9]:
grouped = df.groupby(['vid_title'])['author_name'].apply(list)
new = pd.DataFrame({'videos':grouped.index, 'authors':grouped.values})
new

,videos,authors
0,BEST JOBS in Data Science,"[guiseppehammer, fernleaf1, p s, Lambda Citize..."
1,Become a DATA ANALYST with NO degree?!? The Go...,"[R, El_Shaddai Fire Ministries, South India., ..."
2,Building a bot to scrape job data… How NOT to ...,"[Laxya Berde, Cara Ziegel, redmash beats, Stor..."
3,Calculated Fields in Tableau (Formulas & IF St...,"[Anas El-Sahly, Mustapha El Ouahabi]"
4,Certificates vs Degree for Data Analysts (ft. ...,"[Orlando, Get Things Built, S E, Rema Khiangte..."
...,...,...
60,Use THIS to showcase EXPERIENCE in Data Science,"[Germano Vieira, dinhquy, Andrew, Shams Shaikh..."
61,Using VS Code with Python for Data Science / D...,"[yevgeniy Bondar, Georgina Zhou, BXJacky, Pabl..."
62,What is Business Intelligence (BI)? #shorts,[Mercy Dlaba]
63,Why you should NOT be a Data Analyst,"[sskylark _, Daniel Hallam, TheSweetAmy0, Hami..."


Nella seguente lista vengono definiti i titoli dei video appartenenti unicamente alle 2 playlist menzionate sopra.

In [10]:
python=["Python for Data Science / Analysis ft. 'The Office' Dataset - P.0",
'Python Objects frequently used in Data Science / Data Analysis - P.1',
'Python If Statements for Data Science / Data Analysis - P.2',
'Python For & While Loops for Data Science / Data Analysis - P.3',
'Python List Comprehension for Data Science / Data Analysis - P.4',
'Python Functions for Data Science / Data Analysis - P.5',
'Lambda Functions for Data Science / Data Analysis - Python P.6',
'Install Python for Data Science on Mac & Windows (PC) with Anaconda - P.1',
'How to run Python for Data Science - Editors vs IDEs - P.2',
'Install VS Code with Python for Data Science / Data Analysis - P.3',
'Understanding Virtual Environments for Data Science / Data Analysis - P.4',
'Using VS Code with Python for Data Science / Data Analysis - P.5']

Successivamente, all'interno del dataset *new* vengono selezionati solo i titoli dei video contenuti nella lista *python*.

In [11]:
new = new[new["videos"].isin(python)]
new = new.reset_index()
new

,index,videos,authors
0,25,How to run Python for Data Science - Editors v...,"[Muzammil Abbas, Teck Jiun Ho, Jordan Hubbard,..."
1,27,Install Python for Data Science on Mac & Windo...,"[Bobby _, Jonathan Jarzabek, Musaab Shawesh, A..."
2,28,Install VS Code with Python for Data Science /...,"[Syed Sirajudeen, Kok Yang Chuang, Nisais N, K..."
3,30,Lambda Functions for Data Science / Data Analy...,"[Abhishek Kanani, Angie H, Ben Odams, Bryan Go..."
4,40,Python For & While Loops for Data Science / Da...,"[shubham negi, adfbi, Learner Learner, Дмитрий..."
5,41,Python Functions for Data Science / Data Analy...,"[Daniel Mateo, Toni torre, Rafael Tenório, Ank..."
6,42,Python If Statements for Data Science / Data A...,"[JohnPaul Bertiz, Thayná Azevedo Carvalho, adf..."
7,43,Python List Comprehension for Data Science / D...,"[Terris Thompson, Flying Thestrals, Irfan Ulla..."
8,44,Python Objects frequently used in Data Science...,"[adfbi, Cris Olivares, Eminent Jade, Willian F..."
9,58,Understanding Virtual Environments for Data Sc...,"[yevgeniy Bondar, Eddie Brito, Pablo Dini, Jen..."


Si procede dunque con la costruzione del grafo, come spiegato in precedenza:

In [24]:
G_new = nx.Graph() 
for i in range(len(new)):
  gruppo = list(itertools.combinations(new.loc[i].authors, 2))
  for k in gruppo:
    if (k[0], k[1]) in G_new.edges():
      data = G_new.get_edge_data(k[0], k[1]) #, key='edge')
      G_new.add_edge(k[0], k[1], weight=data['weight']+1)
    else:
      G_new.add_edge(k[0], k[1], weight=1)

In [22]:
print(len(list(G_new.edges)), len(list(G_new.nodes)))

702 96


Il grafo è costituito da 96 nodi e 702 archi.

É possibile visualizzare il grafo in una pagina *html*:

In [23]:
net_new = Network('500px', '500px')
net_new.from_nx(G_new)
# net.show_buttons(filter_=['physics'])
net_new.show('G_new.html')

Si prosegue ora con un'analisi legata alla *Community Detection* utilizzando l'approccio Girvan-Newman, già descritto in precedenza:

In [15]:
result = nx.algorithms.community.centrality.girvan_newman(G_new) 
communities = next(result)
len(communities)

5

In [18]:
communities_len = []
for i in range(len(communities)):
  communities_len.append(len(list(communities)[i]))
print(communities_len) # numerosità delle communities

[48, 6, 6, 28, 8]


In [19]:
G_communities = G_new # grafo con le communities
for el in range(len(communities)):
  for i in range(len(list(communities[el]))):
    G_communities.nodes[list(communities[el])[i]]['group'] = el

In [20]:
net_communities = Network('500px', '500px')
net_communities.from_nx(G_communities)
# net.show_buttons(filter_=['physics'])
net_communities.show('G_communities.html') # plot del grafo con le communities

Dalla analisi emergono 5 diverse comunità, 2 caratterizzate da una numerosità molto più elevata delle restanti 3.

Infine, vengono calcolati i valori di modulartità ed assortatività della rete riguardante i video Python:

In [16]:
modularity = nx.algorithms.community.quality.modularity(G_new, communities=communities)
modularity

0.4774911285048971

Il valore di modularità è pari a 0.48 circa; questo suggerisce che la configuranzione a cluster trovata, si distingue da una possibile composizione randomica della rete. Inoltre, essendo il valore positivo, si evincono dei legami positivi all'interno della rete (ovvero il numero di archi presenti nella rete è maggiore del numero di archi attesi).

In [17]:
assortativity = nx.algorithms.assortativity.degree_assortativity_coefficient(G_new)
assortativity

0.48193197398331

Anche il valore di assortatività della rete è positivo. Grazie a questo è possibile evincere che la rete sia assortativa.